<a href="https://colab.research.google.com/github/Mounish2023/handsonllm_learning/blob/main/text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install --q transformers Datasets sentence-transformers openai

In [7]:
from datasets import DatasetDict, Dataset, load_dataset

data = load_dataset("rotten_tomatoes")
data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

In [4]:
data.shape

{'train': (8530, 2), 'validation': (1066, 2), 'test': (1066, 2)}

In [12]:
data['train'][:5]

{'text': ['the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .',
  'the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson\'s expanded vision of j . r . r . tolkien\'s middle-earth .',
  'effective but too-tepid biopic',
  'if you sometimes like to go to the movies to have fun , wasabi is a good place to start .',
  "emerges as something rare , an issue movie that's so honest and keenly observed that it doesn't feel like one ."],
 'label': [1, 1, 1, 1, 1]}

In [14]:
positive_data = data['train'].filter(lambda ex: ex['label']==1)
positive_data.shape

(4265, 2)

5 examples of positive reviews in train dataset

In [16]:
positive_data[:5]

{'text': ['the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .',
  'the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson\'s expanded vision of j . r . r . tolkien\'s middle-earth .',
  'effective but too-tepid biopic',
  'if you sometimes like to go to the movies to have fun , wasabi is a good place to start .',
  "emerges as something rare , an issue movie that's so honest and keenly observed that it doesn't feel like one ."],
 'label': [1, 1, 1, 1, 1]}

In [17]:
negative_data = data['train'].filter(lambda ex: ex['label']==0)
negative_data

Filter:   0%|          | 0/8530 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label'],
    num_rows: 4265
})

5 examples of Negative reviews in train dataset

In [18]:
negative_data[:5]

{'text': ['simplistic , silly and tedious .',
  "it's so laddish and juvenile , only teenage boys could possibly find it funny .",
  'exploitative and largely devoid of the depth or sophistication that would make watching such a graphic treatment of the crimes bearable .',
  '[garbus] discards the potential for pathological study , exhuming instead , the skewed melodrama of the circumstantial situation .',
  'a visually flashy but narratively opaque and emotionally vapid exercise in style and mystification .'],
 'label': [0, 0, 0, 0, 0]}